<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/ClubRaiders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import datetime
from named.Faction import Faction
from named.FactionInstance import FactionInstance
from named.Station import Station
from named.InhabitedSystem import InhabitedSystem

import Club

import pandas as pd
import json
import datetime
from named.Faction import Faction
from named.FactionInstance import FactionInstance
from named.Station import Station
from named.InhabitedSystem import InhabitedSystem

import Club

all_factions_dict: Faction = {}
# all_stations_dict : Station = {}
all_systems_dict: InhabitedSystem = {}
club_factions_dict: Faction = {}
club_systems_dict: FactionInstance = {}

#
# Identified Pro-Club Faction Names
#


with open("data/factions.jsonl", 'r') as handle:
    for line in handle:
        lCurFaction = json.loads(line)
        lCurFactionId = int(lCurFaction[ 'id' ])
        curFaction = Faction(lCurFaction)
        all_factions_dict[ lCurFactionId ] = curFaction
        if Club.proClubFaction(curFaction):
            club_factions_dict[ lCurFactionId ] = curFaction

# with open(stations_file, 'r') as handle:
#      for line in handle:
#         station = json.loads(line)
#         system_id = station[ SYSTEM_ID ]
#         current_station = stations_dict.get(system_id)

with open("data/systems_populated.jsonl", 'r') as handle:
    for line in handle:
        sys = json.loads(line)
        tid = int(sys[ 'id' ])
        foo = InhabitedSystem(sys)
        all_systems_dict[ tid ] = foo


def desiredState(state_dict):
    for state in state_dict:
        if (state[ 'id' ] == 64): return 64
        if (state[ 'id' ] == 65): return 65
        if (state[ 'id' ] == 73): return 73
        if (state[ 'id' ] == 96): return 96
        if (state[ 'id' ] == 104): return 104

    return 0


for tsys in all_systems_dict.values():
    ttsys: InhabitedSystem = tsys
    mfp = ttsys.getMinorFactionPresences()
    for faction_ptr in mfp:
        if faction_ptr is None:
            continue
        faction_id = int(faction_ptr[ 'minor_faction_id' ])
        if faction_id is None:
            continue
        if faction_id in club_factions_dict:
            fac = club_factions_dict[ faction_id ]
            facname = fac.get_name2()
            if (facname.startswith("*")): continue  # filters player factions

            sysname = ttsys.get_name()
            facHome = fac.get_homesystem_id()
            sysid = ttsys.get_id()
            vulnerable = True
            if (sysid == facHome): vulnerable = False

            govt = ttsys.getGovernment()
            # allg = fac.get_allegiance()

            inf = faction_ptr[ 'influence' ]
            # sinf = '{:04.2f}'.format(inf)
            # print(sinf)

            # updated = ttsys.getUpdated();
            # date = datetime.datetime.utcfromtimestamp(updated)
            # ds = date.strftime("%m/%d/%Y %H:%M:%S")

            active_states = json.dumps(faction_ptr[ 'active_states' ])
            hasWar = desiredState(faction_ptr[ 'active_states' ])
            if (govt == "Anarchy"):
                hasWar = -16
            if (hasWar == 0 and inf <= 3.5 and inf > 0.0):
                hasWar = -15
            if (hasWar == 104 and inf > 10.0):
                hasWar = 0

            sysIns = FactionInstance(fac, ttsys, inf, hasWar)
            club_systems_dict[ faction_id ] = sysIns
            #print(facname + "," + sysname)
            #print("=====================================================================================")
            # print(facname + "," + sysname + "," + x + "," + y + "," + z + "," + allg + "," + sinf + "," + war+ "," + ds )  # + "," + allg)

for xcs in club_systems_dict.values():
    cs: FactionInstance = xcs
    vulnerable = cs.isVulnerable()
    if (vulnerable == False): continue
    cs.printCSV()

Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.60,Low Influence,19-May-2020 22:21
Ekono Emperor's Grace,Ekono,59.12,-155.16,99.81,Empire,1.50,Low Influence,20-May-2020 01:03
Kitche Emperor's Grace,Phi Gruis,6.72,-106.34,44.59,Empire,3.50,Low Influence,13-May-2020 02:29
Sirius Luxury Transports,Col 285 Sector BQ-N c7-13,158.31,69.50,-42.03,Independent,13.90,Anarchy,15-May-2020 14:47
Salaallang Emperor's Grace,Salaallang,58.88,-175.28,-62.88,Empire,13.30,Election,19-May-2020 16:26
Aracenus Emperor's Grace,Aracenus,111.31,-65.88,68.00,Empire,8.59,Civil War,19-May-2020 00:27
Asase Ya Emperor's Grace,Lambda Horologii,93.69,-129.12,22.19,Empire,12.51,War,20-May-2020 01:35
Sirius Corporation,Te Kaha,-172.81,13.31,-154.62,Independent,28.20,Anarchy,20-May-2020 01:27
Dainayiman Emperor's Grace,Xiancheim,137.56,-96.94,28.44,Empire,12.40,Election,19-May-2020 22:04
Botaiti Emperor's Grace,Botaiti,111.03,-143.69,-6.53,Empire,13.19,Election,16-May-2020 22:52
Sirius Hyperspace,Wolf 202,

In [2]:
from IPython.display import HTML, display
import tabulate

arr = []

for xcs in club_systems_dict.values():
    cs: FactionInstance = xcs
    vulnerable = cs.isVulnerable()
    if (vulnerable == False): continue
    arr.append(cs.asArray())

tabulate.tabulate(arr, tablefmt='html')

Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.6,Low Influence,19-May-2020 22:21
Ekono Emperor's Grace,Ekono,59.12,-155.16,99.81,Empire,1.5,Low Influence,20-May-2020 01:03
Kitche Emperor's Grace,Phi Gruis,6.72,-106.34,44.59,Empire,3.5,Low Influence,13-May-2020 02:29
Sirius Luxury Transports,Col 285 Sector BQ-N c7-13,158.31,69.5,-42.03,Independent,13.9,Anarchy,15-May-2020 14:47
Salaallang Emperor's Grace,Salaallang,58.88,-175.28,-62.88,Empire,13.3,Election,19-May-2020 16:26
Aracenus Emperor's Grace,Aracenus,111.31,-65.88,68,Empire,8.59,Civil War,19-May-2020 00:27
Asase Ya Emperor's Grace,Lambda Horologii,93.69,-129.12,22.19,Empire,12.51,War,20-May-2020 01:35
Sirius Corporation,Te Kaha,-172.81,13.31,-154.62,Independent,28.2,Anarchy,20-May-2020 01:27
Dainayiman Emperor's Grace,Xiancheim,137.56,-96.94,28.44,Empire,12.4,Election,19-May-2020 22:04
Botaiti Emperor's Grace,Botaiti,111.03,-143.69,-6.53,Empire,13.19,Election,16-May-2020 22:52
Sirius Hyperspace,Wolf 202,-70.16,-27.59,-140.22,Independent,7.21,Anarchy,20-May-2020 01:09


In [3]:
arr = []

for xcs in club_systems_dict.values():
    cs: FactionInstance = xcs
    arr.append(cs.asArray())

tabulate.tabulate(arr, tablefmt='html')

Vodyanes Emperor's Grace,Vodyanes,80.22,-55.66,-14.47,Empire,34.75,-,19-May-2020 17:45
Worster Insurance,64 Ceti,-37.06,-100.06,-79.12,Independent,2.6,Low Influence,19-May-2020 22:21
Abroin Universal PLC,HIP 72726,-103.72,101.66,-38.5,Independent,6.51,-,03-May-2020 04:40
HIP 5183 Emperor's Grace,HIP 5183,33.25,-230.91,4.38,Empire,23.9,-,19-May-2020 18:16
Rishair Emperor's Grace,Rishair,95.72,-160.47,40.47,Empire,6.11,-,19-May-2020 21:08
Umaspi Emperor's Grace,Vivata,17.97,-77.38,-17.59,Empire,34.9,-,19-May-2020 02:33
Ekono Emperor's Grace,Ekono,59.12,-155.16,99.81,Empire,1.5,Low Influence,20-May-2020 01:03
HIP 101031 Emperor's Grace,Teno,57.28,-129.94,153.41,Empire,7.7,-,20-Apr-2020 20:21
Hodack Prison Colony,Vucub Huan,65.31,35.03,52.25,Independent,63.66,-,20-May-2020 02:08
Bill Turner,Alioth,-33.66,72.47,-20.66,Alliance,0,-,20-May-2020 02:30
Turner Research Group,Mel 22 Sector GM-V c2-8,-192.12,-194.97,-471.66,Alliance,100,-,18-May-2020 18:23
